## Eastern Demand Gathering Demo, V5
Authors: Bainan, Vicky, Dan M.

Last updated: 11/14/19

In [1]:
import pandas as pd
import numpy as np
import requests as r
import json
import pickle
import matplotlib as plt
from collections import Counter, defaultdict
from prereise.gather.demanddata.eia import download_map_demand, find_fix_outliers, get_eia_data, fill_missing_demand
from prereise.gather.demanddata.eia import transform_demand_to_region, map_grid_buses_to_county, transform_ba_to_loadzone
from powersimdata.input.grid import Grid
import json

BAs are associated with abbreviations. Demand data from EIA is imported using an API Key.
Note that data for some BAs will not be found (i.e., YAD) and this will be reported by the download_map_demand.get_BA_demand function.

In [2]:
eastern_ba = ['SWPP','MISO','AECI','SPA','EEI','LGEE','OVEC','PJM','NYIS','ISNE','TVA','CPLW','DUK','YAD','CPLE','AEC','SOCO','SEPA','SC','TAL','SCEG','GVL','JEA','SEC','FPC','NSB','TEC','FMPP','FPL','HST']

In [3]:
start = pd.to_datetime('2016-01-01 00:00:00')
end = pd.to_datetime('2016-12-31 23:00:00')

eastern_demand = download_map_demand.get_BA_demand(eastern_ba, start, end, 'd826fe7862968708eb6a3c2639d6da1b')
eastern_demand_2016 = eastern_demand.loc[start:end]
eastern_demand_2016

EEI not found
YAD not found
SEPA not found


,SWPP,MISO,AECI,SPA,LGEE,OVEC,PJM,NYIS,ISNE,TVA,...,SCEG,GVL,JEA,SEC,FPC,NSB,TEC,FMPP,FPL,HST
2016-01-01 00:00:00+00:00,30875.0,78481.0,2917.0,68.0,4280.0,NaN,95072.0,19726.0,16063.0,18742.0,...,2637.0,270.0,NaN,175.0,7389.0,65.0,2969.0,2507.0,18036.0,84.0
2016-01-01 01:00:00+00:00,31457.0,77902.0,2946.0,83.0,4245.0,NaN,92298.0,19008.0,15273.0,18555.0,...,2558.0,258.0,NaN,164.0,6931.0,61.0,2778.0,2386.0,16990.0,79.0
2016-01-01 02:00:00+00:00,30969.0,75821.0,2910.0,58.0,4078.0,NaN,89416.0,18340.0,14568.0,18180.0,...,2487.0,244.0,NaN,152.0,0.0,53.0,2575.0,2226.0,15650.0,73.0
2016-01-01 03:00:00+00:00,30397.0,73863.0,2878.0,80.0,3935.0,NaN,86496.0,17557.0,13850.0,17744.0,...,2374.0,229.0,NaN,140.0,5830.0,51.0,2397.0,2049.0,14406.0,66.0
2016-01-01 04:00:00+00:00,29818.0,71919.0,2829.0,69.0,3793.0,NaN,83405.0,16772.0,13108.0,17306.0,...,2256.0,213.0,NaN,128.0,5405.0,48.0,2223.0,1908.0,13389.0,60.0
2016-01-01 05:00:00+00:00,29085.0,69589.0,2779.0,63.0,3669.0,NaN,80685.0,16055.0,12737.0,16731.0,...,2127.0,197.0,NaN,117.0,4969.0,43.0,2091.0,1760.0,12411.0,56.0
2016-01-01 06:00:00+00:00,28343.0,67929.0,2707.0,68.0,3650.0,80.0,78661.0,15250.0,12503.0,16298.0,...,2014.0,184.0,1181.0,107.0,4617.0,40.0,1960.0,1636.0,12043.0,53.0
2016-01-01 07:00:00+00:00,27803.0,66212.0,2681.0,70.0,3566.0,80.0,77817.0,14583.0,12241.0,15975.0,...,1941.0,176.0,1128.0,100.0,4328.0,37.0,1859.0,1566.0,11393.0,52.0
2016-01-01 08:00:00+00:00,27483.0,64903.0,2643.0,70.0,3484.0,80.0,76287.0,14013.0,11824.0,15660.0,...,1852.0,165.0,1067.0,92.0,4018.0,34.0,1752.0,1480.0,10680.0,50.0
2016-01-01 09:00:00+00:00,27229.0,64091.0,2653.0,65.0,3484.0,80.0,75546.0,13632.0,11616.0,15533.0,...,1780.0,159.0,1025.0,87.0,3819.0,32.0,1682.0,1424.0,10172.0,47.0


### Load in subarea profile information for MISO and SPP

In [4]:
miso_subarea_demand = pd.read_csv('miso_subarea_demand_profile_2016.csv',index_col=0)
spp_subarea_demand = pd.read_csv('spp_subarea_demand_profile_2016.csv',index_col=0)
spp_subarea_demand.index = eastern_demand_2016.index

Proportionately distribute overall eia demand for MISO among subregions

In [5]:
frac_subarea_miso = miso_subarea_demand.copy()
for zone in miso_subarea_demand.columns:
    frac_subarea_miso[zone] = miso_subarea_demand[zone].divide(miso_subarea_demand.sum(axis=1)).multiply(eastern_demand_2016['MISO'])

Proportionately distribute overall eia demand for SPP among subregions

In [6]:
frac_subarea_spp = spp_subarea_demand.copy()
for zone in spp_subarea_demand.columns:
    frac_subarea_spp[zone] = spp_subarea_demand[zone].divide(spp_subarea_demand.sum(axis=1)).multiply(eastern_demand_2016['SWPP'])

Concatenate MISO and SPP subarea columns to existing Eastern demand

In [7]:
eastern_demand_2016 = pd.concat([eastern_demand_2016, miso_subarea_demand, spp_subarea_demand],axis=1)

Some BAs are very small. In these cases, BAs are agregated into BA areas as follows:

In [8]:
BA_area ={
        'ISNE':['ISNE'],
        'NYIS':['NYIS'],
        'PJM':['PJM','OVEC'],
        'AEC':['AEC'],
        'SOCO':['SOCO'],
        'TVA_LGEE':['TVA','LGEE'],
        'Florida':['TAL','JEA','GVL','SEC','FPC','NSB','FMPP','TEC','FPL','HST'],
    
        'AECI':['AECI'],

        #'MISO':['MISO','AECI'],
    
        'LRZ1': ['LRZ1'],
        'LRZ2_7': ['LRZ2_7'],
        'LRZ3_5': ['LRZ3_5'],
        'LRZ4': ['LRZ4'],
        'LRZ6': ['LRZ6'],
        'LRZ8_9_10': ['LRZ8_9_10'],
    
        #'SWPP':['SWPP','SPA'],
        'CSWS': ['CSWS','SPA'],
        'EDE': ['EDE'],
        'GRDA': ['GRDA'],
        'KACY': ['KACY'],
        'KCPL_INDN': ['KCPL_INDN'],
        'LES': ['LES'],
        'MPS': ['MPS'],
        'NPPD': ['NPPD'],
        'OKGE': ['OKGE'],
        'OPPD': ['OPPD'],
        'SECI': ['SECI'],
        'SPRM': ['SPRM'],
        'SPS': ['SPS'],
        'WAUE': ['WAUE'],
        'WFEC': ['WFEC'],
        'WR': ['WR'],

        'Carolina':['CPLW','DUK','SCEG','SC','CPLE']
        }


Use adjacent demand data to fill missing values using a series of rules:
1. Monday: look forward one day
2. Tues - Thurs: average of look forward one day and look back one day
3. Fri: look back one day
4. Sat: look forward one day
5. Sun: look back one day

If data is still missing after applying the above rules, week ahead and week behind data is used
1. Monday: look forward two days 
2. Tues: look forward two days
3. Wed: average of look forward two days and look back two days
4. Thurs: look back two days
5. Fri: look back two days
6. Sat - Sun: average of look back one week and look forward one week

If data is still missing after applying the above rules, week ahead and week behind data is used:
1. Mon - Sun: average of look back one week and look forward one week

In [9]:
filled_eastern_demand_2016 = fill_missing_demand.replace_with_shifted_demand(eastern_demand_2016, start, end)

C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\prereise\gather\demanddata\eia\fill_missing_demand.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  shifted_demand_BA[baName] = fill_ba_demand(shifted_demand_BA, baName, dayMap)
C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\prereise\gather\demanddata\eia\fill_missing_demand.py:31: Se

In [10]:
for baName in filled_eastern_demand_2016.columns:
    print(baName)
    print(filled_eastern_demand_2016.loc[filled_eastern_demand_2016[baName].isnull()])

SWPP
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST, LRZ1, LRZ2_7, LRZ3_5, LRZ4, LRZ6, LRZ8_9_10, CSWS, EDE, GRDA, KACY, KCPL_INDN, LES, MPS, NPPD, OKGE, OPPD, SECI, SPRM, SPS, WAUE, WFEC, WR]
Index: []

[0 rows x 49 columns]
MISO
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST, LRZ1, LRZ2_7, LRZ3_5, LRZ4, LRZ6, LRZ8_9_10, CSWS, EDE, GRDA, KACY, KCPL_INDN, LES, MPS, NPPD, OKGE, OPPD, SECI, SPRM, SPS, WAUE, WFEC, WR]
Index: []

[0 rows x 49 columns]
AECI
Empty DataFrame
Columns: [SWPP, MISO, AECI, SPA, LGEE, OVEC, PJM, NYIS, ISNE, TVA, CPLW, DUK, CPLE, AEC, SOCO, SC, TAL, SCEG, GVL, JEA, SEC, FPC, NSB, TEC, FMPP, FPL, HST, LRZ1, LRZ2_7, LRZ3_5, LRZ4, LRZ6, LRZ8_9_10, CSWS, EDE, GRDA, KACY, KCPL_INDN, LES, MPS, NPPD, OKGE, OPPD, SECI, SPRM, SPS,

In [11]:
eastern_agg_demand = transform_demand_to_region.transform_ba_to_region(eastern_demand_2016, BA_area)
eastern_agg_demand

ISNE regional demand was summed from ['ISNE']

NYIS regional demand was summed from ['NYIS']

PJM regional demand was summed from ['OVEC', 'PJM']

AEC regional demand was summed from ['AEC']

SOCO regional demand was summed from ['SOCO']

TVA_LGEE regional demand was summed from ['TVA', 'LGEE']

Florida regional demand was summed from ['FMPP', 'GVL', 'TAL', 'FPC', 'SEC', 'HST', 'FPL', 'TEC', 'NSB', 'JEA']

AECI regional demand was summed from ['AECI']

LRZ1 regional demand was summed from ['LRZ1']

LRZ2_7 regional demand was summed from ['LRZ2_7']

LRZ3_5 regional demand was summed from ['LRZ3_5']

LRZ4 regional demand was summed from ['LRZ4']

LRZ6 regional demand was summed from ['LRZ6']

LRZ8_9_10 regional demand was summed from ['LRZ8_9_10']

CSWS regional demand was summed from ['SPA', 'CSWS']

EDE regional demand was summed from ['EDE']

GRDA regional demand was summed from ['GRDA']

KACY regional demand was summed from ['KACY']

KCPL_INDN regional demand was summed from ['KCPL_I

,ISNE,NYIS,PJM,AEC,SOCO,TVA_LGEE,Florida,AECI,LRZ1,LRZ2_7,...,NPPD,OKGE,OPPD,SECI,SPRM,SPS,WAUE,WFEC,WR,Carolina
2016-01-01 00:00:00+00:00,16063.0,19726.0,95072.0,492.0,23927.0,23022.0,31808.0,2917.0,12863.13,19590.03,...,1715.894,3302.916,1255.451,654.778,331.677,3340.971,3247.225,1089.707,3118.536,22878.0
2016-01-01 01:00:00+00:00,15273.0,19008.0,92298.0,488.0,24347.0,22800.0,29950.0,2946.0,12644.95,19112.33,...,1702.234,3257.226,1232.957,644.269,325.933,3303.212,3204.807,1087.542,3061.272,22097.0
2016-01-01 02:00:00+00:00,14568.0,18340.0,89416.0,465.0,23574.0,22258.0,21262.0,2910.0,12224.99,18441.55,...,1690.372,3225.992,1213.319,639.428,321.829,3289.910,3174.050,1072.959,3024.605,21357.0
2016-01-01 03:00:00+00:00,13850.0,17557.0,86496.0,444.0,22762.0,21679.0,25439.0,2878.0,11868.49,17831.50,...,1693.390,3232.956,1206.674,634.397,320.809,3277.186,3160.193,1088.153,3015.726,20466.0
2016-01-01 04:00:00+00:00,13108.0,16772.0,83405.0,429.0,21857.0,21099.0,23625.0,2829.0,11525.23,17238.04,...,1709.054,3264.036,1214.124,633.686,324.849,3264.970,3164.260,1092.147,3043.398,19469.0
2016-01-01 05:00:00+00:00,12737.0,16055.0,80685.0,410.0,20974.0,20400.0,21878.0,2779.0,11103.99,16538.11,...,1743.495,3336.751,1240.155,640.722,331.891,3293.560,3204.534,1092.157,3108.199,18412.0
2016-01-01 06:00:00+00:00,12503.0,15250.0,78741.0,394.0,20711.0,19948.0,22040.0,2707.0,10911.12,15824.53,...,1791.892,3426.342,1268.296,653.800,342.400,3338.410,3235.650,1109.800,3188.136,17505.0
2016-01-01 07:00:00+00:00,12241.0,14583.0,77897.0,366.0,20505.0,19541.0,20847.0,2681.0,10560.67,15271.17,...,1835.476,3474.677,1279.637,667.021,343.015,3370.079,3285.148,1129.020,3235.700,16968.0
2016-01-01 08:00:00+00:00,11824.0,14013.0,76367.0,348.0,20051.0,19144.0,19535.0,2643.0,10211.46,14792.03,...,1831.122,3466.161,1269.686,669.237,338.973,3376.680,3284.290,1119.453,3223.521,16444.0
2016-01-01 09:00:00+00:00,11616.0,13632.0,75626.0,341.0,19761.0,19017.0,18638.0,2653.0,9977.08,14462.12,...,1816.366,3443.504,1280.955,676.113,337.946,3403.783,3266.272,1103.302,3236.765,16108.0


In [12]:
eastern_agg_demand.to_pickle("./eastern_agg_demand.pkl")

Indentify and fix outliers. Outliers defined as demand hours for which the ramp rate (difference between two hours) is greater than 3 standard deviations of the ramp rates for the BA across all hours in the year. 
Demand data for hours identified as outliers are replaced with the average of the neighboring non-outlier hour's demand.
Stretches of >5 consecutive outlier hours are flagged.

In [13]:
eastern_fix_outliers = find_fix_outliers.fix_dataframe_outliers(eastern_agg_demand)


ISNE

NYIS

PJM
1357 104873.0 104873.0
1358 137033.0 104409.0
1359 103945.0 103945.0
1734 67360.0 67360.0
1735 67.0 66320.0
1736 65280.0 65280.0
1756 76763.0 76763.0
1757 52.0 52.0
1780 49.0 49.0
1781 74584.0 74584.0
7468 70188.0 70188.0
7469 0.0 69438.5
7470 68689.0 68689.0

AEC

SOCO
2410 19206.0 19206.0
2411 26343.0 19742.0
2412 20278.0 20278.0

TVA_LGEE
701 23378.0 23378.0
702 4517.0 22913.0
703 22448.0 22448.0
704 22324.0 22324.0
705 3964.0 22365.0
706 22406.0 22406.0
1735 14850.0 14850.0
1736 2638.0 2638.0
1757 2897.0 2897.0
1758 15124.0 15124.0
7469 16362.0 16362.0
7470 4106.0 15976.0
7471 15590.0 15590.0
8237 25897.0 25897.0
8238 5649.0 25647.5
8239 25398.0 25398.0
8477 28120.0 28120.0
8478 7081.0 27567.5
8479 27015.0 27015.0

Florida
1564 18918.0 18918.0
1565 28308.0 18278.0
1566 17638.0 17638.0
1876 31575.0 31575.0
1877 21050.0 21050.0
3912 33901.0 33901.0
3913 16354.0 16354.0
3916 13220.0 13220.0
3917 24792.0 24792.0
4194 42510.0 42510.0
4195 21544.0 21544.0
4204 15518.0 15

In [14]:
eastern_fix_outliers 

,ISNE,NYIS,PJM,AEC,SOCO,TVA_LGEE,Florida,AECI,LRZ1,LRZ2_7,...,NPPD,OKGE,OPPD,SECI,SPRM,SPS,WAUE,WFEC,WR,Carolina
2016-01-01 00:00:00+00:00,16063.0,19726.0,95072.0,492.0,23927.0,23022.0,31808.0,2917.0,12863.13,19590.03,...,1715.894,3302.916,1255.451,654.778,331.677,3340.971,3247.225,1089.707,3118.536,22878.0
2016-01-01 01:00:00+00:00,15273.0,19008.0,92298.0,488.0,24347.0,22800.0,29950.0,2946.0,12644.95,19112.33,...,1702.234,3257.226,1232.957,644.269,325.933,3303.212,3204.807,1087.542,3061.272,22097.0
2016-01-01 02:00:00+00:00,14568.0,18340.0,89416.0,465.0,23574.0,22258.0,21262.0,2910.0,12224.99,18441.55,...,1690.372,3225.992,1213.319,639.428,321.829,3289.910,3174.050,1072.959,3024.605,21357.0
2016-01-01 03:00:00+00:00,13850.0,17557.0,86496.0,444.0,22762.0,21679.0,25439.0,2878.0,11868.49,17831.50,...,1693.390,3232.956,1206.674,634.397,320.809,3277.186,3160.193,1088.153,3015.726,20466.0
2016-01-01 04:00:00+00:00,13108.0,16772.0,83405.0,429.0,21857.0,21099.0,23625.0,2829.0,11525.23,17238.04,...,1709.054,3264.036,1214.124,633.686,324.849,3264.970,3164.260,1092.147,3043.398,19469.0
2016-01-01 05:00:00+00:00,12737.0,16055.0,80685.0,410.0,20974.0,20400.0,21878.0,2779.0,11103.99,16538.11,...,1743.495,3336.751,1240.155,640.722,331.891,3293.560,3204.534,1092.157,3108.199,18412.0
2016-01-01 06:00:00+00:00,12503.0,15250.0,78741.0,394.0,20711.0,19948.0,22040.0,2707.0,10911.12,15824.53,...,1791.892,3426.342,1268.296,653.800,342.400,3338.410,3235.650,1109.800,3188.136,17505.0
2016-01-01 07:00:00+00:00,12241.0,14583.0,77897.0,366.0,20505.0,19541.0,20847.0,2681.0,10560.67,15271.17,...,1835.476,3474.677,1279.637,667.021,343.015,3370.079,3285.148,1129.020,3235.700,16968.0
2016-01-01 08:00:00+00:00,11824.0,14013.0,76367.0,348.0,20051.0,19144.0,19535.0,2643.0,10211.46,14792.03,...,1831.122,3466.161,1269.686,669.237,338.973,3376.680,3284.290,1119.453,3223.521,16444.0
2016-01-01 09:00:00+00:00,11616.0,13632.0,75626.0,341.0,19761.0,19017.0,18638.0,2653.0,9977.08,14462.12,...,1816.366,3443.504,1280.955,676.113,337.946,3403.783,3266.272,1103.302,3236.765,16108.0


## Get bus to BA area mapping via county
Author: Bainan  
Last updated: 11/14/19

First, get Eastern grid. 

In [14]:
eastern = Grid(['Eastern'])

--> Loading Eastern interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


Warning: the following step takes around 3 hours to finish. The results are stored in bus_ba_map.csv for future use, as implemented below:

In [15]:
#bus_ba_map, bus_no_BA_match = map_grid_buses_to_county(eastern)*

#bus_ba_map.to_csv('bus_ba_map.csv')*

The csv file is created at this stage to aviod time-intensive reruns.

In [20]:
bus_ba_map = pd.read_csv('bus_ba_map.csv',index_col=0)

The following step loads a file that stores list of counties in each BA area territory. The file is created mannually based on external references.

In [22]:
data = json.load(open('EasternBAtoCountyDraft6.txt'))
ba_county_list = {}
for val in data['groups'].values():
    ba_county_list[val['label']] = set(val['paths'])

In [ ]:
for index,row in bus_ba_map.iterrows():
    for BA,clist in ba_county_list.items():
        try:
            county = row['County'].replace(' ','_')
            county = county.replace('.','')
            county = county.replace('-','')
            county = county.replace('\'','_')
            if row['County'] == 'LaSalle__IL':
                county = 'La_Salle__IL'
            if row['County'] == 'Lac Qui Parle__MN':
                county = 'Lac_qui_Parle__MN'
            if row['County'] == 'Baltimore__MD':
                county = 'Baltimore_County__MD'
            if row['County'] == 'District of Columbia__DC':
                county = 'Washington__DC'
            if row['County'] == 'St. Louis City__MO':
                county = 'St_Louis_Co__MO'
            if county in clist:
                bus_ba_map.loc[index,'BA'] = BA
                break
        except:
            continue

In [ ]:
bus_ba_map

Check if there are buses where BA is empty/not found, which is due to bus being outside of United States. These were fixed manually by assigning to the nearest county. This procedure is commented out as it was used just one time. 

Check if there are any buses where BA is null, which is due to a name mis-match.

Then check if there are buses where county is null. 

In [23]:
#bus_no_BA_match = list(bus_ba_map[~bus_ba_map['BA'].astype(bool)].index)

bus_no_BA_match = list(bus_ba_map[bus_ba_map['BA'].isna()].index)
bus_no_BA_match

[]

In [ ]:
bus_no_county_match = list(bus_ba_map[bus_ba_map['County'].isna()].index)
bus_no_county_match

Add zone name into the data frame for reference.

In [24]:
bus_ba_map.loc[:,'zone_name'] = eastern.bus[eastern.bus['Pd']>0]['zone_id'].apply(lambda x: eastern.id2zone[x])

Fix mismatch county names in Virginia and Maryland

In [25]:
for ind in bus_no_BA_match:
    if bus_ba_map.loc[ind,'zone_name'] in {'Virginia Mountains','West Virginia','Virginia Tidewater','Maryland'}:
        bus_ba_map.loc[ind,'BA'] = 'PJM'

## Manually assign outliers (outside US territory) to the nearest BA
Bus no county match: [91: ISNE, 7991: NYIS, 7992: NYIS, 8707: NYIS, 8708: NYIS, 40644: MISO]  
Assign the rest no-ba-match buses to SWPP.  
Assign buses in ERCOT Texas to SWPP or MISO based on the location by observation.  
Make BA code cosnistent with EIA data source.  
Overwrite the final results with two new columns for each row (bus): BA and zone_name. 

In [26]:
# Manually assign outliers (outside US territory) to the nearest BA
# bus no county match: [91: ISNE, 7991: NYIS, 7992: NYIS, 8707: NYIS, 8708: NYIS, 40644: MISO]
bus_ba_map.loc[91,'BA'] = 'ISNE'
bus_ba_map.loc[7991,'BA'] = 'NYIS'
bus_ba_map.loc[7992,'BA'] = 'NYIS'
bus_ba_map.loc[8707,'BA'] = 'NYIS'
bus_ba_map.loc[8708,'BA'] = 'NYIS'
bus_ba_map.loc[40644,'BA'] = 'MISO'

# Assign the rest no-ba-match buses to SWPP
for ind in bus_no_BA_match:
    bus_ba_map.loc[ind,'BA'] = 'SWPP'

# Assign buses in ERCOT Texas to SWPP or MISO based on the location by observation
miso_tx_ind = bus_ba_map[(bus_ba_map['BA']=='ERCOT Texas') & (bus_ba_map['zone_name']=='East Texas') & ((bus_ba_map['County']=='Montgomery__TX') | (bus_ba_map['County']=='Walker__TX'))].index
for ind in bus_ba_map[bus_ba_map['BA']=='ERCOT Texas'].index:
    if ind in miso_tx_ind:
        bus_ba_map.loc[ind,'BA']='MISO'
    else:
        bus_ba_map.loc[ind,'BA']='SWPP'

# Make BA Code consistent with EIA data source
ba_code_fix = {'ISONE':'ISNE','NYISO':'NYIS'}
bus_ba_map.replace(ba_code_fix,inplace=True)

# The csv file is overwritten by the final results with two new columns for each row (bus): BA and zone_name.
bus_ba_map.to_csv('bus_ba_map.csv')

### Generate final demand profile

Read in ba region to bus mapping files

In [15]:
bus_map = pd.read_csv('bus_ba_map.csv',index_col=0)
bus_map_spp = pd.read_csv('bus_ba_map_spp.csv',index_col=0)
bus_map_miso = pd.read_csv('MISO_mapping.csv',index_col=0)
# bus_map.set_index('bus_id')

Make MISO subarea mapping dataframe consistent with others

In [17]:
bus_map_miso['BA'] = bus_map_miso['miso_zone']
bus_map_miso.drop(columns=['miso_zone','state'],inplace=True)

Remove MISO and SWPP bus from orignal mapping and concatenate with mappings of corresponding subregions

In [18]:
no_miso_spp_bus_map = bus_map[~((bus_map['BA']=='MISO')|(bus_map['BA']=='SWPP'))]
bus_map_new = pd.concat([no_miso_spp_bus_map,bus_map_miso,bus_map_spp]).sort_index()

Transform to load zone using the demand that has missing data filled and outliers fixed, prepared from the first steps in the demo notebook.

In [21]:
final_eastern_demand =  transform_ba_to_loadzone.map_to_loadzone(eastern_fix_outliers, bus_map_new)
final_eastern_demand

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,Alabama,Florida Panhandle,Arkansas,Illinois Downstate,Missouri East,East Texas,Kansas,Louisiana,Missouri West,Oklahoma,...,Mississippi,New Jersey,Pennsylvania Eastern,Pennsylvania Western,Delaware,Maryland,Virginia Tidewater,West Virginia,New Mexico Eastern,Texas Panhandle
2016-01-01 00:00:00+00:00,8786.661262,2157.601335,5609.234444,5398.507754,8442.423107,2053.281718,5090.381964,10364.560424,2799.802680,7324.420005,...,6504.524092,11711.598199,11124.355240,5559.094083,1457.475255,9771.440984,9705.345710,3013.381864,682.274722,2521.486776
2016-01-01 01:00:00+00:00,8858.455490,2139.145162,5629.147755,5253.425202,8444.570018,2057.605091,5020.140452,10426.890757,2780.207467,7260.266222,...,6513.077046,11365.838316,10799.704824,5396.817214,1414.949208,9486.330991,9422.164237,2925.457751,674.563787,2492.989426
2016-01-01 02:00:00+00:00,8590.193974,1891.201541,5538.274214,5122.741612,8264.520589,2025.313285,4983.199993,10250.778705,2763.677844,7182.376885,...,6383.247589,11008.828130,10462.450517,5228.263066,1370.767497,9190.120825,9127.957674,2834.110493,671.847326,2482.950184
2016-01-01 03:00:00+00:00,8310.425066,1940.190500,5461.446469,4973.190807,8110.782717,2004.478728,4988.265153,10050.233252,2774.022306,7224.904497,...,6235.035657,10644.098537,10120.700774,5057.431124,1326.003236,8890.005042,8829.871913,2741.558794,669.248901,2473.347168
2016-01-01 04:00:00+00:00,8014.225846,1844.411223,5388.380567,4831.735011,7961.086881,1987.183452,5039.297926,9839.067900,2805.046867,7298.178554,...,6080.018765,10259.253684,9758.957691,4876.617844,1278.617508,8572.313986,8514.329760,2643.587117,666.754216,2464.127548
2016-01-01 05:00:00+00:00,7706.016459,1751.448018,5307.554645,4803.095550,7703.573022,1970.273144,5143.416810,9587.499430,2858.656335,7437.854844,...,5895.339694,9919.856439,9440.621130,4717.493169,1236.919293,8292.754079,8236.660832,2557.374576,672.592709,2485.704900
2016-01-01 06:00:00+00:00,7576.658639,1739.522536,5253.102946,4642.111658,7509.837514,1963.599989,5277.514060,9379.326270,2932.272146,7618.786934,...,5757.399595,9668.988931,9212.970110,4603.613314,1207.117333,8092.950969,8038.209215,2495.757966,681.751729,2519.553947
2016-01-01 07:00:00+00:00,7454.113674,1699.324364,5202.774469,4446.551460,7422.326195,1954.905013,5351.485671,9206.928276,2967.338825,7739.227948,...,5641.002933,9551.067249,9113.988533,4554.006129,1194.178622,8006.205174,7952.050180,2469.006722,688.218998,2543.455071
2016-01-01 08:00:00+00:00,7283.356828,1641.818548,5187.432708,4349.687491,7325.716674,1954.886759,5335.511938,9136.720988,2940.933115,7744.486522,...,5566.014246,9355.428235,8934.847865,4464.411603,1170.723376,7848.952727,7795.861408,2420.512168,689.567018,2548.436957
2016-01-01 09:00:00+00:00,7192.832095,1603.761276,5159.975843,4324.180085,7269.121277,1947.951322,5350.053250,9063.587622,2899.355136,7708.873076,...,5520.157388,9257.700936,8848.039512,4420.965037,1159.363679,7772.793208,7720.217042,2397.025589,695.101844,2568.892045


Replace datetime index with one compatible with PowerSimData

In [22]:
index_reference = pd.read_csv('spp_subarea_demand_profile_2016.csv',index_col=0)
final_eastern_demand.index = index_reference.index

In [23]:
final_eastern_demand.to_csv('eastern_demand_v6.csv')